In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy.sparse.linalg import svds
from functools import partial
from scipy.sparse import csc_matrix
from sklearn.neighbors import kneighbors_graph
from tqdm.auto import tqdm

/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv("train.csv", sep=",")
data['rating'] = data['rating'].astype(float)
training_data1, validation_data = train_test_split(data, random_state= 2,test_size=0.1)

In [4]:
user_X_movie = training_data1.pivot_table(index='customer-id', columns='movie-id', values='rating')
user_X_movie_zeros = user_X_movie.fillna(np.nan)
print(user_X_movie_zeros.head)


<bound method NDFrame.head of movie-id     1      2      3      4      5      6      8      11     12     \
customer-id                                                                  
6              NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
7              NaN    NaN    NaN    NaN    NaN    NaN    5.0    NaN    NaN   
8              NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
10             NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
25             NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...            ...    ...    ...    ...    ...    ...    ...    ...    ...   
33152          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
33154          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
33155          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
33160          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
33182          NaN    NaN    NaN  

In [5]:


def emsvd(Y, k=None, tol=1E-3, maxiter=None):


    if k is None:
        svdmethod = partial(np.linalg.svd, full_matrices=False)
    else:
        svdmethod = partial(svds, k=k)
    if maxiter is None:
        maxiter = np.inf
  # initialize the missing values to their respective column means
    mu_hat = np.nanmean(Y, axis=0, keepdims=1)
    #mu_hat=0
    valid = np.isfinite(Y)
    Y_hat = np.where(valid, Y, mu_hat)

    halt = False
    ii = 1
    v_prev = 0

    while not halt:

        # SVD on filled-in data
        U, s, Vt = svdmethod(Y_hat - mu_hat)

        # impute missing values
        Y_hat[~valid] = (U.dot(np.diag(s)).dot(Vt) + mu_hat)[~valid]

        # update bias parameter
        mu_hat = Y_hat.mean(axis=0, keepdims=1)

        # test convergence using relative change in trace norm
        v = s.sum()
        if ii >= maxiter or ((v - v_prev) / v_prev) < tol:
            halt = True
        ii += 1
        v_prev = v

    return Y_hat, mu_hat, U, s, Vt

In [6]:
svdres, lol, users, lol1, lol2=emsvd(user_X_movie_zeros)

In [ ]:
neighbours = kneighbors_graph(users,n_neighbors=10, mode='distance', metric='minkowski')

In [ ]:
# print(svdres.shape)

In [ ]:
print(svdres.round())

[[4. 5. 4. ... 3. 2. 3.]
 [4. 5. 4. ... 3. 2. 3.]
 [4. 5. 4. ... 3. 2. 3.]
 ...
 [4. 5. 4. ... 3. 2. 3.]
 [4. 5. 4. ... 3. 2. 3.]
 [4. 5. 4. ... 3. 2. 3.]]


In [ ]:
# cos_sim = cosine_similarity(svdres)
# cos_sim = 2 - cos_sim 
# for i in range(cos_sim.shape[0]):
#     cos_sim[i][i] = 0

# cos_sim = csc_matrix(cos_sim)


In [ ]:
# print(cos_sim.shape)

(5903, 5903)


In [ ]:
# print(cos_sim.min())

0.0


In [ ]:
#cos_sim=cos_sim+0.000000000001

In [ ]:
number_of_clusters = 3
print("\n\START BUILDING CLUSTERS\n\n")
sc = SpectralClustering(n_clusters=number_of_clusters,n_components=10,n_neighbors=10,random_state=40,affinity='precomputed_nearest_neighbors', n_init=100, assign_labels='kmeans').fit(neighbours)
labels = sc.labels_


\START BUILDING CLUSTERS




In [ ]:
print(len(labels))

5901


In [ ]:
print("\n\CLUSTERS BUILT\n\n")
cluster_map = pd.DataFrame()
cluster_map['customer_id'] = user_X_movie_zeros.index.values
cluster_map['cluster_no'] = labels


\CLUSTERS BUILT




In [ ]:
cluster_map.rename(columns = {'customer_id':'customer-id'}, inplace = True)

print("Saving cluster map")
cluster_map.to_csv('clusterMap.csv')

Saving cluster map


In [ ]:
training_data_with_labels = pd.merge(training_data1, cluster_map, on='customer-id')

In [ ]:
for i in range(number_of_clusters):
    rows_from_same_cluster = training_data_with_labels[training_data_with_labels['cluster_no']==i]
    rows_from_same_cluster.to_csv('clustered_records_'+str(i)+'.csv',index=False)

In [ ]:

for i in range(3):
    clustered_records = pd.read_csv("clustered_records_"+str(i)+".csv")
    unique_m_ids = clustered_records['movie-id'].unique()
    clustered_records['mean_rating'] = 0
    
    for mid in unique_m_ids:
        intermediate_rating = clustered_records[clustered_records['movie-id']==mid]['rating']
        
        intermediate_rating[intermediate_rating!=0]
        mean_rating=4

        if(intermediate_rating.mean() > 5):
            mean_rating=5
        elif(intermediate_rating.mean() < 1):
            mean_rating=1
        else:
             mean_rating = round(intermediate_rating.mean())

        clustered_records.loc[clustered_records['movie-id']==mid, 'mean_rating'] = mean_rating
        
    clustered_records['mean_rating'] = clustered_records['mean_rating'].astype(int)
    clustered_records.to_csv('clustered_records_'+str(i)+'.csv',index=False)





CREATING CSVs




CREATED ALL CSVs




In [ ]:
c0 = pd.read_csv("clustered_records_0.csv")
c1 = pd.read_csv("clustered_records_1.csv")
c2 = pd.read_csv("clustered_records_2.csv")
# c3 = pd.read_csv("clustered_records_3.csv")
# c4 = pd.read_csv("clustered_records_4.csv")




final_search_cluster = c0.append([c1,c2]) #,c3,c4])
final_search_cluster.to_csv("final_search_cluster.csv")



/tmp/ipykernel_3154856/3725127420.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_search_cluster = c0.append([c1,c2]) #,c3,c4])




STARTING TESTs




In [ ]:

validation_data_clean_actual = pd.concat([validation_data['movie-id'], validation_data['customer-id'], validation_data['rating']], axis=1)
validation_data_clean_to_predict = pd.concat([validation_data['movie-id'], validation_data['customer-id'], validation_data['rating']], axis=1)



In [ ]:
recno = 0
for index, row in tqdm(validation_data_clean_to_predict.iterrows()):
    recno+=1
    movie_id, customer_id = row['movie-id'], row['customer-id']
    try:
        cluster_number = cluster_map[cluster_map['customer-id'] == customer_id]['cluster_no'].iloc[0]

        mean_rating = final_search_cluster[final_search_cluster['movie-id']==movie_id]['mean_rating'].iloc[0]
    except:

        mean_rating = 3

  
    validation_data_clean_to_predict.at[index,'mean_rating'] = mean_rating

validation_data_clean_to_predict.to_csv("validation_predictions.csv")

print("validation_data_clean_to_predict : \n",validation_data_clean_to_predict.head(5))

mse = mean_squared_error(validation_data_clean_actual['rating'], validation_data_clean_to_predict['mean_rating'])
print("MSE : ",mse)


100000it [02:54, 572.76it/s]


validation_data_clean_to_predict : 
         movie-id  customer-id  rating  mean_rating
309190      9491        29948     4.0          4.0
240953      9233        11583     4.0          4.0
609687     11283        13017     4.0          4.0
757042      1145        18122     3.0          3.0
788110     15156        10593     4.0          4.0
MSE :  1.14443


In [ ]:
lol=validation_data_clean_to_predict['mean_rating']
lollist=lol.tolist()

for i in range(len(lollist)):
    lollist[i]=5


In [ ]:
test_df = pd.read_csv("test.csv", names=['movie_id','customer-id','rating', 'date'])

test_df['rating']=0
print(test_df.head)

recno = 0
for index, row in test_df.iterrows():
    recno+=1
    movie_id, customer_id = row['movie_id'], row['customer-id']
   
    
    try:
        cluster_number = cluster_map[cluster_map['customer-id'] == customer_id]['cluster_no'].iloc[0]
        mean_rating = final_search_cluster[final_search_cluster['movie-id']==movie_id]['mean_rating'].iloc[0]
        
        
    except:
        mean_rating = 3

    test_df.at[index,'mean_rating'] = mean_rating
    print("mean_rating = ",mean_rating, "\t #record = ",recno)

print("test_df : \n",test_df.head(5))

test_df.to_csv("SPECTRAL_TEST_PREDICTIONS.csv")
test_rating = test_df['mean_rating'].tolist()
with open('spectral_prediction_results.txt', 'w') as file:
    for result in test_rating:
        file.write('%d\n' % result)

NameError: name 'pd' is not defined